### Importing Necessary Libraries:

In [ ]:
import tensorflow as tf
import keras
from keras import datasets, layers, models
from keras.layers import Input, Lambda, Dense, Flatten,Conv2D, MaxPooling2D, Activation, Dropout
from keras import models
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

#### Defining Data Path:

In [ ]:
#Defining Data Path

train_path = 'C:/Users/Administrator/Desktop/PPP/DataSet2/Training/'
valid_path = 'C:/Users/Administrator/Desktop/PPP/DataSet2/Validation/'
#test_path = 'C:/Users/Administrator/Desktop/PPP/DataSet2/Testing/'

#### Data generator with TF:

In [ ]:
# Data generator with TF

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode = "rgb")
"""
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            color_mode = "rgb",
                                            shuffle=False
                                            
                                           )
"""""

valid_set = test_datagen.flow_from_directory(valid_path,
                                                        target_size=(224,224),
                                                        batch_size=32,
                                                        class_mode='categorical',
                                                        color_mode = "rgb",
                                                        shuffle=False
                                                        )

### Printing some samples:

In [ ]:
# Data Samples 

for _ in range(5):
    img, label = training_set.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

### Building the model (Sequential approrach):

In [ ]:
# Building the model (Sequential approrach)

model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()


### Model Compilation and training:

In [ ]:
# Model Compilation

model.compile(
  loss='categorical_crossentropy',
  optimizer='Adam',
  metrics=['accuracy']
)

In [ ]:
# Training 

with tf.device('/GPU:0'):
    history = model.fit(train_set, epochs=30, 
                        validation_data=valid_set,
                        steps_per_epoch=len(train_set),
                        validation_steps=len(valid_set),
                        batch_size = 64)

### Model assessment and printing some metrics:

In [ ]:
# Model training history visualization

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("Cnn.h5")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("Cnn.h5")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Confusion Matrix Output

Y_pred = model.predict_generator(test_set, 1705 // 32+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['Glioma', 'meningioma', 'no_Tumor', 'pituitary']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


target= ['Glioma', 'meningioma', 'no_Tumor', 'pituitary']

# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

y_pred = model.predict_generator(test_set, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
print('ROC AUC score:', multiclass_roc_auc_score(test_set.classes, y_pred))


c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()


In [ ]:
# Some Optional visualization
import seaborn as sns
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_set)

print(valid_accuracy)